In [0]:
import codecs
from gensim.models import word2vec

corpus = []

with codecs.open("/content/drive/My Drive/all_maeshori_train_utf8.txt", "r", "utf-8") as f:
    text = f.read().splitlines()
for sen in text:
  temp = sen.split()
  corpus.append(temp[1:])

#モデルを作る
#sg=1ならskip-gram
#size:単語ベクトルの次元数
#window:ウインドウサイズ5
#negative:ネガティブサンプリング数
#alpha:学習率
#min_count:設定した頻度　デフォルトだと頻度が5未満の文字は無視されるようだ
#batch_words:学習時のバッチサイズ
print("learn")
model = word2vec.Word2Vec(corpus, sg=1, size=200, window=5,negative=10,alpha=0.005,batch_words=10,min_count=1)
print("save")
model.save('/content/drive/My Drive/all_maeshori_train_word2vec2_utf8.model')
print("save done")



In [0]:
import codecs
from gensim.models import word2vec

model = word2vec.Word2Vec.load('/content/drive/My Drive/all_maeshori_train_word2vec2_utf8.model')

word = "bony"

# 単語のベクトルを見る
try:
  word_vector = model.wv[word]
  print(word_vector)
except KeyError as error:
  print(error)

# 尾張と類似している単語を見る
#similar_words = model.wv.most_similar(positive=[word], topn=9)
#print(*[" ".join([v, str("{:.2f}".format(s))]) for v, s in similar_words], sep="\n")

In [0]:
from gensim.models import word2vec
import numpy as np
import codecs

model = word2vec.Word2Vec.load('/content/drive/My Drive/all_maeshori_train_word2vec2_utf8.model')

def avg_feature_vector(sentence, model, num_features):
    words = sentence.split()
    feature_vec = np.zeros(num_features, dtype="float") # 特徴ベクトルの入れ物を初期化
    for word in words:
        feature_vec = np.add(feature_vec, model[word]) #行列の足し算
    if len(words) > 0:
        feature_vec = np.divide(feature_vec, len(words)) #文章の単語数で割り算
    return feature_vec

#print(avg_feature_vector("subject make ghostscript work organization lehigh university lines need font files get also advice get cica nice interface ghostscript ghostscript user interface makes user friendly using interface get copy windows directory copy gui executables files ghostscript directory line set ghostscript gui directory ready use enjoy steve brewer files need download ghostscript never used ghostscript files downloaded cica unfortunately seem work needs files want run ghostscript windows pc understand versions environments files need download get info steve brewer", model, 200))
#print(model.wv['subject'])
#print(model.wv['make'])
#sentence = "subject make ghostscript work organization lehigh university lines need font files get also advice get cica nice interface ghostscript ghostscript user interface makes user friendly using interface get copy windows directory copy gui executables files ghostscript directory line set ghostscript gui directory ready use enjoy steve brewer files need download ghostscript never used ghostscript files downloaded cica unfortunately seem work needs files want run ghostscript windows pc understand versions environments files need download get info steve brewer"

doc_vec = []
label = []

with codecs.open("/content/drive/My Drive/all_maeshori_train_utf8.txt", "r", "utf-8") as f:
    text = f.read().splitlines()
for s in text:
  s = s.split()
  cat = s[0]
  sen = s[1:]
  sen = ' '.join(sen)
  doc_vec.append(avg_feature_vector(sen, model, 200))
  label.append(cat)


#print(doc_vec)
#print(label)

np.save('/content/drive/My Drive/np_save',doc_vec)





In [0]:
from sklearn import svm
from sklearn.externals import joblib
import codecs

# コストパラメータやRBFカーネルパラメータはgrid_searchによって最適なパラメータを選択
svm = svm.SVC(C = 10, gamma = 0.1, kernel = 'rbf')
# train_docはベクトル化した文書のリスト
# labelsは文書のカテゴリのリスト

train_doc = np.load('/content/drive/My Drive/np_save.npy')
labels = []

with codecs.open("/content/drive/My Drive/all_maeshori_train_utf8.txt", "r", "utf-8") as f:
    text = f.read().splitlines()
for s in text:
  s = s.split()
  cat = s[0]
  labels.append(cat)

svm.fit(train_doc, labels)
#分類器の保存
joblib.dump(svm, '/content/drive/My Drive/svm.pkl.cmp', compress = True)



In [0]:
import numpy as np
a = np.load('/content/drive/My Drive/np_save.npy')
print(type(a))
print(a.shape)

print(a)